In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns",999)

In [2]:
def all_values(df):
    for i in df.columns:
        print("")
        print(i,":", df[i].value_counts(),"\n")

### Ideas
* Instead of horoscope sign, just do "seriousness" of horoscopes
  * DONE
* Religion column and "faith intensity" column? "" with horoscope
  * DONE
* Diet can be bifurcated ('stricts', 'mostlys', everything else)
  * DONE
* For SPEAKS - maybe English == English (fluent) Y/N, and then anything else fluent for a few languages?
  * DONE
* Word count, or verb count, for all essay columns
* Should income be categorical or continuous?
* Should I put NaNs in for all incomes of -1 (did not answer?). It's 48000+ observations...
* Education: I think I want to group with highest level of achievement AND make groupings for current vs non-current student

### Simple, basic transformations
* **Drop** essays, last_online, drugs, drinks, diet, religion, speaks, location, ethnicity, offspring, sign
*

# Helper Functions

In [36]:
def number_of_uniques(df):
    for i in df.columns:
        print(i,":", len(df[i].unique()))

# Mandatory Cleaning

In [303]:
cupid = pd.read_csv('okcupid/profiles.csv')

In [304]:
### Once ready for modeling, I should certainly drop:

# cupid.drop(columns=['last_online','location'

In [305]:
# Creating new columns
cupid['diet_intensity'] = cupid.diet.str.extract('(\w+ )')
cupid['diet_choice'] = cupid.diet.str.split(" ").str[-1]
cupid['primary_ethnicity'] = cupid.ethnicity.str.replace(", ",",").str.split(",",expand=True)[0]
cupid['future_kids_views'] = cupid.offspring.str.extract('(\S*\w*\swant.*)', expand=True)[0].value_counts()
cupid['has_kids'] = cupid.offspring.str.extract('((\S+\s|^)(have|has)(\s+\S+|$))', expand=True)[0] 
cupid['likes_cats'] = cupid.pets.str.contains("(^|\s+)likes cats", regex=True)
cupid['likes_dogs'] = cupid.pets.str.contains("(^|\s+)likes dogs", regex=True)
cupid['dislikes_cats'] = cupid.pets.str.contains("dislikes cats")
cupid['dislikes_dogs'] = cupid.pets.str.contains("dislikes dogs")
cupid['has_cats'] = cupid.pets.str.contains("has cats")
cupid['has_dogs'] = cupid.pets.str.contains("has dogs")
cupid['english_fluent'] = cupid.speaks.str.strip().str.extract("(english($|,|\s\(f))")[0]
cupid['english_poor'] = cupid.speaks.str.extract("(english \([po])")[0]
cupid['spanish_fluent'] =cupid.speaks.str.contains("spanish \(f", regex=True)
cupid['spanish_not_poorly'] = cupid.speaks.str.contains("spanish \([po]",regex=True)
cupid['signs_fun'] = cupid.sign.str.extract("(fun to think about)|(but it)|(matters a lot)")[0]
cupid['signs_unimportant'] = cupid.sign.str.extract("(fun to think about)|(but it)|(matters a lot)")[1]
cupid['signs_important'] = cupid.sign.str.extract("(fun to think about)|(but it)|(matters a lot)")[2]
cupid['religion_unserious'] = cupid.religion.str.extract("(not too serious)|(laughing)|(somewhat)|(very serious)")[0]
cupid['religion_laughing'] =cupid.religion.str.extract("(not too serious)|(laughing)|(somewhat)|(very serious)")[1]
cupid['religion_somewhat'] = cupid.religion.str.extract("(not too serious)|(laughing)|(somewhat)|(very serious)")[2]
cupid['religion_serious'] =cupid.religion.str.extract("(not too serious)|(laughing)|(somewhat)|(very serious)")[3]
cupid['religion_name'] = cupid.religion.str.extract("(\w+)")

/home/spencer/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  import sys
/home/spencer/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [313]:
cupid.religion_name.value_counts()

agnosticism     8812
other           7743
atheism         6985
christianity    5787
catholicism     4758
judaism         3098
buddhism        1948
hinduism         450
islam            139
Name: religion_name, dtype: int64

In [314]:
number_of_uniques(cupid)

age : 54
body_type : 13
diet : 19
drinks : 7
drugs : 4
education : 33
essay0 : 54351
essay1 : 51517
essay2 : 48636
essay3 : 43534
essay4 : 49261
essay5 : 48964
essay6 : 43604
essay7 : 45555
essay8 : 39325
essay9 : 45444
ethnicity : 218
height : 61
income : 13
job : 22
last_online : 30123
location : 199
offspring : 16
orientation : 3
pets : 16
religion : 46
sex : 2
sign : 49
smokes : 6
speaks : 7648
status : 5
diet_intensity : 3
diet_choice : 7
primary_ethnicity : 10
future_kids_views : 1
has_kids : 6
likes_cats : 3
likes_dogs : 3
dislikes_cats : 3
dislikes_dogs : 3
has_cats : 3
has_dogs : 3
english_fluent : 4
english_poor : 3
spanish_fluent : 3
spanish_not_poorly : 3
signs_fun : 2
signs_unimportant : 2
signs_important : 2
religion_unserious : 2
religion_laughing : 2
religion_somewhat : 2
religion_serious : 2
religion_name : 10


In [306]:
cupid.columns

Index(['age', 'body_type', 'diet', 'drinks', 'drugs', 'education', 'essay0',
       'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7',
       'essay8', 'essay9', 'ethnicity', 'height', 'income', 'job',
       'last_online', 'location', 'offspring', 'orientation', 'pets',
       'religion', 'sex', 'sign', 'smokes', 'speaks', 'status',
       'diet_intensity', 'diet_choice', 'primary_ethnicity',
       'future_kids_views', 'has_kids', 'likes_cats', 'likes_dogs',
       'dislikes_cats', 'dislikes_dogs', 'has_cats', 'has_dogs',
       'english_fluent', 'english_poor', 'spanish_fluent',
       'spanish_not_poorly', 'signs_fun', 'signs_unimportant',
       'signs_important', 'religion_unserious', 'religion_laughing',
       'religion_somewhat', 'religion_serious', 'religion_name'],
      dtype='object')

In [311]:
cupid.education.str.extract("(college)").value_counts()

True     34485
False    18833
Name: education, dtype: int64

In [309]:
cupid.education.value_counts()

graduated from college/university    23959
graduated from masters program        8961
working on college/university         5712
working on masters program            1683
graduated from two-year college       1531
graduated from high school            1428
graduated from ph.d program           1272
graduated from law school             1122
working on two-year college           1074
dropped out of college/university      995
working on ph.d program                983
college/university                     801
graduated from space camp              657
dropped out of space camp              523
graduated from med school              446
working on space camp                  445
working on law school                  269
two-year college                       222
working on med school                  212
dropped out of two-year college        191
dropped out of masters program         140
masters program                        136
dropped out of ph.d program            127
dropped out

In [115]:
cupid['primary_ethnicity'] = cupid.ethnicity.str.replace(", ",",").str.split(",",expand=True)[0]

In [116]:
cupid.head()

,age,body_type,diet,drinks,drugs,education,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9,ethnicity,height,income,job,last_online,location,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status,diet_intensity,diet_choice,primary_ethnicity
0,22,a little extra,strictly anything,socially,never,working on college/university,about me:<br />\n<br />\ni would love to think...,currently working as an international agent fo...,making people laugh.<br />\nranting about a go...,"the way i look. i am a six foot half asian, ha...","books:<br />\nabsurdistan, the republic, of mi...",food.<br />\nwater.<br />\ncell phone.<br />\n...,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet!<br />\nyou...,"asian, white",75.0,-1,transportation,2012-06-28-20-30,"south san francisco, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism and very serious about it,m,gemini,sometimes,english,single,strictly,anything,asian
1,35,average,mostly other,often,sometimes,working on space camp,i am a chef: this is what that means.<br />\n1...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories.<br /...,NaN,NaN,i am very open and will share just about anyth...,NaN,white,70.0,80000,hospitality / travel,2012-06-29-21-41,"oakland, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism but not too serious about it,m,cancer,no,"english (fluently), spanish (poorly), french (...",single,mostly,other,white
2,38,thin,anything,socially,NaN,graduated from masters program,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement<br />\nconversation<br />\ncreation<b...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ...",NaN,68.0,-1,NaN,2012-06-27-09-10,"san francisco, california",NaN,straight,has cats,NaN,m,pisces but it doesn&rsquo;t matter,no,"english, french, c++",available,NaN,anything,NaN
3,23,thin,vegetarian,socially,NaN,working on college/university,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . .<br />\nlynch, j...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.,white,71.0,20000,student,2012-06-28-14-22,"berkeley, california",doesn&rsquo;t want kids,straight,likes cats,NaN,m,pisces,no,"english, german (poorly)",single,NaN,vegetarian,white
4,29,athletic,NaN,socially,never,graduated from college/university,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at:<br />\nhttp://bag...,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians<br />\nat the...",NaN,NaN,NaN,NaN,NaN,"asian, black, other",66.0,-1,artistic / musical / writer,2012-06-27-21-26,"san francisco, california",NaN,straight,likes dogs and likes cats,NaN,m,aquarius,no,english,single,NaN,NaN,asian


In [94]:
cupid.diet_intensity.value_counts()

mostly       21508
strictly      6704
Name: diet_intensity, dtype: int64

In [93]:
cupid.diet_choice.value_counts()

anything      27881
vegetarian     4986
other          1790
vegan           702
kosher          115
halal            77
Name: diet_choice, dtype: int64

In [70]:
cupid.diet.str.split(" ")#, expand=True)

0          [strictly, anything]
1               [mostly, other]
2                    [anything]
3                  [vegetarian]
4                           NaN
5            [mostly, anything]
6          [strictly, anything]
7            [mostly, anything]
8          [strictly, anything]
9            [mostly, anything]
10           [mostly, anything]
11           [mostly, anything]
12                          NaN
13           [mostly, anything]
14           [mostly, anything]
15         [strictly, anything]
16                          NaN
17                          NaN
18         [mostly, vegetarian]
19           [mostly, anything]
20           [mostly, anything]
21                          NaN
22           [mostly, anything]
23            [strictly, vegan]
24         [strictly, anything]
25                   [anything]
26           [mostly, anything]
27                   [anything]
28                          NaN
29                          NaN
                  ...          
59916   

In [64]:
cupid['diet'].str.rsplit(" ", expand=True)



# cupid.loc[len(cupid['diet'].str.split(" ")) == 2, 'diet_intensity'] = cupid.diet.str.split(" ")[0]
# cupid.loc[len(cupid['diet'].str.split(" ")) == 1, 'diet_intensity'] = "None"


# for i in cupid.diet.str.split(" "):
#     if i == 2:
#         cupid['diet_intensity'] = cupid.diet.str.split(" ")[0]
#     elif i ==1:
#         cupid['diet_intensity'] = "None"
#     else:
#         cupid['diet_intensity'] = np.nan


,0,1
0,strictly,anything
1,mostly,other
2,anything,None
3,vegetarian,None
4,NaN,NaN
5,mostly,anything
6,strictly,anything
7,mostly,anything
8,strictly,anything
9,mostly,anything


In [ ]:
df.loc[df['A'] == df['B'], 'C'] = 0
df.loc[df['A'] > df['B'], 'C'] = 1
df.loc[df['A'] < df['B'], 'C'] = -1

In [54]:
cupid.diet_intensity.value_counts()

Series([], Name: diet_intensity, dtype: int64)

In [ ]:
cupid['diet_intensity'] = 

In [41]:
cupid.diet.str.startswith('mostly')

0        False
1         True
2        False
3        False
4          NaN
5         True
6        False
7         True
8        False
9         True
10        True
11        True
12         NaN
13        True
14        True
15       False
16         NaN
17         NaN
18        True
19        True
20        True
21         NaN
22        True
23       False
24       False
25       False
26        True
27       False
28         NaN
29         NaN
         ...  
59916      NaN
59917     True
59918     True
59919      NaN
59920     True
59921      NaN
59922    False
59923    False
59924      NaN
59925     True
59926     True
59927      NaN
59928     True
59929    False
59930    False
59931    False
59932      NaN
59933      NaN
59934    False
59935    False
59936     True
59937    False
59938      NaN
59939      NaN
59940      NaN
59941      NaN
59942     True
59943     True
59944     True
59945      NaN
Name: diet, Length: 59946, dtype: object

In [38]:
cupid.isna().sum()

age                0
body_type       5296
diet           24395
drinks          2985
drugs          14080
education       6628
essay0          5488
essay1          7572
essay2          9638
essay3         11476
essay4         10537
essay5         10850
essay6         13771
essay7         12451
essay8         19225
essay9         12603
ethnicity       5680
height             3
income             0
job             8198
last_online        0
location           0
offspring      35561
orientation        0
pets           19921
religion       20226
sex                0
sign           11056
smokes          5512
speaks            50
status             0
dtype: int64

In [29]:
# print('smokes:',cupid.smokes.unique())
# print('drinks:',cupid.drinks.unique())
# print('drugs:',cupid.drugs.unique())
print('last_online:',cupid.last_online.unique())

last_online: ['2012-06-28-20-30' '2012-06-29-21-41' '2012-06-27-09-10' ...
 '2012-06-02-08-16' '2012-02-17-20-44' '2012-06-14-16-51']


# Optional Cleaning

In [16]:
cupid = cupid.drop(['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9',
                    'last_online', 'drinks', 'drugs', 'diet', 'religion', 'speaks', 'location', 'ethnicity', 'offspring', 'sign'], axis=1)